In [ ]:
import os
from torchvision import datasets,transforms,models
import torch
from torch.utils.data import DataLoader
import ssl
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm


In [ ]:
Base_dir = os.getcwd()

Data_dir=os.path.join(Base_dir,"Data")
train_dir=os.path.join(Data_dir,"train")
validation_dir=os.path.join(Data_dir,"validation")


#extra
#cat_train_dir=os.path.join(train_dir,"Cat")
#dog_train_dir=os.path.join(train_dir,"Dog")

#cat_val_dir=os.path.join(validation_dir,"Cat")
#dog_val_dir=os.path.join(validation_dir,"Dog")


In [16]:
Base_dir = os.getcwd()
print(Base_dir)

c:\Users\VirtualUser\Desktop\NANDANA


In [ ]:
#def count_images(folder,extensions=(".jpg",".png",".jpeg")):
#    return len([
#        file for file in os.listdir(folder)
#        if file.lower().endswith(extensions)
#    ])

#print("Training")
#print("cat:",count_images(cat_train_dir))
#print("dog:",count_images(dog_train_dir))

#print("Validation")
#print("cat:",count_images(cat_val_dir))
#print("dog:",count_images(dog_val_dir))

Training
cat: 9999
dog: 9999
Validation
cat: 2500
dog: 2500


In [ ]:
#required method
train_transform=transforms.Compose([transforms.Resize((224,224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]
                                    ),
                                    transforms.RandomRotation(degrees=15),
                                    
                                    transforms.ColorJitter(brightness=0.2,saturation=0.2,contrast=0.2),
                                    transforms.RandomHorizontalFlip()
                                    ])
val_transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize(
                                        mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]),
                                    transforms.Resize((224,224))
                                
                                ])

In [44]:
train_dataset=datasets.ImageFolder(
    root=train_dir,
    transform=train_transform
)

val_dataset=datasets.ImageFolder(
    root=validation_dir,
    transform=val_transform
)

In [46]:
print(train_dataset.class_to_idx)
print(val_dataset.class_to_idx)

print("train data=",len(train_dataset))
print("validation data=",len(val_dataset))

{'Cat': 0, 'Dog': 1}
{'Cat': 0, 'Dog': 1}
train data= 19998
validation data= 5000


In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=4,
    shuffle=True
)

In [ ]:
val_loader=DataLoader(
    val_dataset,
    batch_size=32,
    num_workers=4,
    shuffle=False
)

In [64]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cpu


In [ ]:
ssl._create_default_https_context=ssl._create_unverified_context

In [ ]:
model=models.resnet50(pretrained=True)

c:\Users\VirtualUser\Desktop\NANDANA\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\VirtualUser\Desktop\NANDANA\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
0.1%

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\VirtualUser/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100.0%


In [70]:
print(model.fc)

Linear(in_features=2048, out_features=1000, bias=True)


In [84]:
for param in model.parameters():
    param.required_grad=False


In [82]:
for name,param in model.named_parameters():
    print(name,param.shape)


conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight torch.Size([256])
layer1.0.bn3.bias torch.Size([256])
layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
layer1.0.downsample.1.weight torch.Size([256])
layer1.0.downsample.1.bias torch.Size([256])
layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer1.1.conv3.weight torch.Size([256, 64, 1, 1])
layer1.1.bn3.weight torch.Size([256])
layer1.1.bn3.bias torch.Size([256])
layer1.2.conv1.weight tor

In [73]:
num_features = model.fc.in_features
print("Number of neurons after feature extraction:", num_features)


Number of neurons after feature extraction: 2048


In [ ]:
#fully connected layer
model.fc=torch.nn.Sequential(
    torch.nn.Linear(num_features,512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(512,2)

)

In [ ]:
criterion=nn.CrossEntropyLoss()

In [ ]:
def get_optimizer(model, lr=0.001):
    optimizer = optim.Adam(
        model.fc.parameters(), 
        lr=lr
    )
    return optimizer


In [79]:
optimizer = get_optimizer(model)


In [ ]:
scheduler = StepLR(
    optimizer,
    step_size=5,
    gamma=0.1
)


In [ ]:
def train_one_epoch(model,train_loader,criterion,optimizer,device):

    model.train()

    total_loss = 0.0
    correct = 0
    total_samples = 0

    for input,labels in tqdm(train_loader,desc="training"):
        input=input.to(device)
        label=labels.to(device)

        optimizer.zero_grad()#clear old gradients

        outputs=model(input)#passing input to model

        loss=criterion(outputs, labels)#loss function

        loss.backward()#backpropagation

        optimizer.step()#update weights based on mistake

        total_loss+=loss.item()#add current loss to total loss

        predicted=torch.max(outputs, 1)#get predicted class

        total_samples+=labels.size(0)#update total counters

        correct+=(predicted==labels).sum().item()#update correct counters

    avg_loss = total_loss/len(train_loader)

    accuracy = (correct/total_samples) * 100

    return avg_loss, accuracy
    

In [ ]:
def validate_one_epoch(model, val_loader, criterion, device):
    
    model.eval()

    total_loss = 0.0
    correct = 0
    total_samples = 0

    
    with torch.no_grad():#Disable gradient calculation
       
        for images, labels in tqdm(val_loader,desc="validation"):
            
            input=input.to(device)
            labels=labels.to(device)

            
            outputs=model(input)

            
            loss=criterion(outputs, labels)#loss function

            total_loss+=loss.item()

           
            predicted = torch.max(outputs, 1)#Get predicted class

           
            total_samples += labels.size(0)#Update accuracy counters
            correct += (predicted == labels).sum().item()

   
    avg_loss = total_loss / len(val_loader)

    accuracy = (correct / total_samples) * 100

    
    return avg_loss, accuracy
